In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, utils
from torch.utils.data import TensorDataset, DataLoader
import torch.backends.cudnn as cudnn
import time
from pathlib import Path
import os

from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist

from quant_mnist_model import *
from QuantModules import *
from _quantUtils import *
from _utils import train, test


%matplotlib inline
%config InlineBackend.figure_format='retina'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True

In [ ]:
(x_train, y_train), (x_test, y_test), min_, max_ = load_mnist()

x_train = np.swapaxes(x_train, 1, 3).astype(np.float32)
x_test = np.swapaxes(x_test, 1, 3).astype(np.float32)

train_dataset = TensorDataset(torch.Tensor(x_train), torch.Tensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=128)

test_dataset = TensorDataset(torch.Tensor(x_test), torch.Tensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=1000)

In [ ]:
# model = classifier().to(device)

# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.0001, weight_decay=1e-4)

# best_prec = 0

for e in range(10):
    print('\nEpoch: %d' % int(e+1))
    train(model, optimizer, train_dataloader)
    prec = test(model, test_dataloader)
    is_best = prec > best_prec
    best_prec = max(prec, best_prec)
    if is_best:
        torch.save(model.state_dict(), 'mnist_2bit.pth')

In [ ]:
model